### Overview
Using pretrained Vision Language Model to finetune zero-shot classifier for low-latency activity data on small model. 
Likely eventually replaced or quantized for mobile device / low-cost compute.

In [ ]:
!pip install -q transformers peft datasets accelerate bitsandbytes


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
import pandas as pd
from datasets import Dataset

# Replace this with your SQL-to-Pandas query
# df = pd.read_sql("SELECT * FROM training_data", con=your_sql_connection)
df = pd.read_csv("your_dataset.csv")  # Example, replace with your dataset

# Process data into Hugging Face Dataset format
def process_data(row):
    return {
        "input": row["input"],
        "label": row["label"],
    }

dataset = Dataset.from_pandas(df)
dataset = dataset.map(process_data)

# Split dataset into training and testing
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model_id = "Qwen/Qwen2-VL-2B"

bnb_config = {
    "load_in_4bit": True,
    "bnb_4bit_use_double_quant": True,
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_compute_dtype": torch.bfloat16,
}

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Add LoRA configurations
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # Modules to apply LoRA
    lora_dropout=0.1,
    task_type="SEQ_CLASSIFICATION"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)


In [ ]:
from transformers import DataCollatorForLanguageModeling

def tokenize_function(example):
    return tokenizer(
        example["input"], truncation=True, padding="max_length", max_length=512
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


### Model Training & Finetuning

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./outputs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    fp16=True,
    push_to_hub=True,  # Enable if you want to push the model to Hugging Face Hub
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()


In [ ]:
# Save the fine-tuned model and tokenizer locally
model.save_pretrained("./finetuned_qwen2")
tokenizer.save_pretrained("./finetuned_qwen2")

print("Model and tokenizer saved locally at './finetuned_qwen2'")


In [ ]:
from datetime import datetime

# Push the fine-tuned model to Hugging Face Hub
model_name = f"finetuned-qwen2-vl-2b-{datetime.now().strftime('%d%H%M')}"
trainer.model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

print(f"Model pushed to Hugging Face Hub under the name '{model_name}'")


### Citations
@article{Qwen2-VL,
  title={Qwen2-VL: Enhancing Vision-Language Model's Perception of the World at Any Resolution}, 
  author={Peng Wang and Shuai Bai and Sinan Tan and Shijie Wang and Zhihao Fan and Jinze Bai and Keqin Chen and Xuejing Liu and Jialin Wang and Wenbin Ge and Yang Fan and Kai Dang and Mengfei Du and Xuancheng Ren and Rui Men and Dayiheng Liu and Chang Zhou and Jingren Zhou and Junyang Lin},
  journal={arXiv preprint arXiv:2409.12191},
  year={2024}
}

@article{Qwen-VL,
  title={Qwen-VL: A Versatile Vision-Language Model for Understanding, Localization, Text Reading, and Beyond},
  author={Bai, Jinze and Bai, Shuai and Yang, Shusheng and Wang, Shijie and Tan, Sinan and Wang, Peng and Lin, Junyang and Zhou, Chang and Zhou, Jingren},
  journal={arXiv preprint arXiv:2308.12966},
  year={2023}
}

https://medium.com/the-ai-forum/instruction-fine-tuning-gemma-2b-on-medical-reasoning-and-convert-the-finetuned-model-into-gguf-844191f8d329
